In [1]:
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
from statsmodels.tsa.api import ExponentialSmoothing
from pmdarima import auto_arima
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Ignore common warnings from statistical models
warnings.filterwarnings("ignore")

# --- 1. Data Loading and Preprocessing (Run Once) ---
def load_and_prepare_data(segments_path=r"C:\Users\Thanish-REVA\Desktop\Reva\Trim-2\Capstone Project -1\Development_Code\phase-2\filtered_segments.csv", 
                          pivot_path=r"C:\Users\Thanish-REVA\Desktop\Reva\Trim-2\Capstone Project -1\Development_Code\phase-2\final_pivot_data.csv"):
    """Loads, cleans, and prepares the data for all models."""
    print("🔄 Step 1: Loading and preparing data...")
    try:
        segments_df = pd.read_csv(segments_path)
        pivot_df = pd.read_csv(pivot_path)
    except FileNotFoundError:
        print("Error: Make sure 'filtered_segments.csv' and 'final_pivot_data.csv' are in the directory.")
        return None

    if 'Unnamed: 0' in pivot_df.columns:
        pivot_df = pivot_df.drop('Unnamed: 0', axis=1)

    target_segments = ['AFX', 'AFY', 'AFZ', 'BFX']
    # Use 'Combined_Segment' as per the provided file structure
    target_parts_df = segments_df[segments_df['Combined_Segment'].isin(target_segments)]
    target_part_numbers = target_parts_df['Part No'].unique()
    
    model_df = pivot_df[pivot_df['Part No'].isin(target_part_numbers)]

    long_df = pd.melt(model_df, id_vars=['Part No'],
                      value_vars=[col for col in model_df.columns if col != 'Part No'],
                      var_name='Date', value_name='Demand')
    
    long_df['Date'] = pd.to_datetime(long_df['Date'])
    long_df.dropna(subset=['Demand'], inplace=True)
    long_df.sort_values(by=['Part No', 'Date'], inplace=True)
    print("✅ Data preparation complete.")
    return long_df

# --- 2. Model Implementations ---

# Model A: XGBoost
def run_xgboost(long_df):
    """Trains XGBoost, evaluates it, and generates a forecast."""
    print("\n--- Running XGBoost Model ---")
    
    # a. Feature Engineering
    def create_features(df):
        df = df.copy()
        df['month'] = df['Date'].dt.month
        df['year'] = df['Date'].dt.year
        df['quarter'] = df['Date'].dt.quarter
        df['lag_1'] = df.groupby('Part No')['Demand'].shift(1)
        df['lag_2'] = df.groupby('Part No')['Demand'].shift(2)
        df['lag_3'] = df.groupby('Part No')['Demand'].shift(3)
        df['rolling_mean_3'] = df.groupby('Part No')['Demand'].shift(1).rolling(window=3, min_periods=1).mean()
        return df

    featured_df = create_features(long_df)
    featured_df.dropna(inplace=True)
    
    # b. Train-Test Split
    split_date = featured_df['Date'].max() - pd.DateOffset(months=3)
    train_df = featured_df[featured_df['Date'] <= split_date]
    test_df = featured_df[featured_df['Date'] > split_date]

    FEATURES = ['month', 'year', 'quarter', 'lag_1', 'lag_2', 'lag_3', 'rolling_mean_3']
    TARGET = 'Demand'
    X_train, y_train = train_df[FEATURES], train_df[TARGET]
    X_test, y_test = test_df[FEATURES], test_df[TARGET]

    # c. Model Training & Evaluation
    reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.01,
                           early_stopping_rounds=50, random_state=42)
    reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    y_pred = reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    metrics = {'RMSE': rmse, 'MAE': mae, 'R-squared': r2}
    
    # d. Forecasting
    last_known_data = featured_df.sort_values('Date').groupby('Part No').tail(1).copy()
    future_dates = pd.date_range(start=long_df['Date'].max() + pd.DateOffset(months=1), periods=3, freq='MS')
    all_forecasts = []

    for part_number in last_known_data['Part No'].unique():
        history_row = last_known_data[last_known_data['Part No'] == part_number].copy()
        for date in future_dates:
            features_for_pred = {'month': date.month, 'year': date.year, 'quarter': date.quarter,
                                 'lag_1': history_row['Demand'].iloc[0], 'lag_2': history_row['lag_1'].iloc[0],
                                 'lag_3': history_row['lag_2'].iloc[0],
                                 'rolling_mean_3': (history_row['Demand'].iloc[0] + history_row['lag_1'].iloc[0] + history_row['lag_2'].iloc[0]) / 3}
            X_future = pd.DataFrame([features_for_pred], columns=FEATURES)
            predicted_demand = reg.predict(X_future)[0]
            all_forecasts.append({'Part No': part_number, 'Forecast': max(0, round(predicted_demand))})
            new_history_data = {'Demand': predicted_demand, 'lag_1': features_for_pred['lag_1'], 'lag_2': features_for_pred['lag_2'], 'lag_3': features_for_pred['lag_3']}
            history_row = pd.DataFrame([new_history_data])
            
    forecast_df = pd.DataFrame(all_forecasts).groupby('Part No')['Forecast'].sum().reset_index()
    forecast_df.rename(columns={'Forecast': '3_Month_Forecast_XGBoost'}, inplace=True)
    
    return forecast_df, metrics

# Model B: SARIMA with fallback
def run_sarima(long_df):
    """Runs SARIMA, evaluates it, and generates a forecast with fallback logic."""
    print("\n--- Running SARIMA Model ---")
    
    # a. Evaluation
    predictions, actuals = [], []
    for part in long_df['Part No'].unique():
        part_df = long_df[long_df['Part No'] == part].set_index('Date')['Demand']
        if len(part_df) < 15 or part_df.var() == 0: continue
        train, test = part_df[:-3], part_df[-3:]
        try:
            model = auto_arima(train, seasonal=True, m=12, stepwise=True, suppress_warnings=True, error_action='ignore')
            preds = model.predict(n_periods=len(test))
            predictions.extend(preds)
            actuals.extend(test)
        except Exception: continue
    
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    metrics = {'RMSE': rmse, 'MAE': mae, 'R-squared': r2}
    
    # b. Forecasting
    all_forecasts = []
    for part in long_df['Part No'].unique():
        part_df = long_df[long_df['Part No'] == part].set_index('Date')['Demand']
        forecast_values = np.array([0, 0, 0])
        try:
            if len(part_df) < 12 or part_df.var() == 0: raise ValueError("Not suitable for SARIMA")
            model = auto_arima(part_df, seasonal=True, m=12, suppress_warnings=True, error_action='raise', stepwise=True)
            forecast_values = model.predict(n_periods=3)
        except Exception:
            if not part_df.empty:
                moving_avg = part_df.tail(6).mean()
                forecast_values = np.array([moving_avg, moving_avg, moving_avg])
        
        total_forecast = sum(max(0, round(val)) for val in forecast_values)
        all_forecasts.append({'Part No': part, '3_Month_Forecast_SARIMA': total_forecast})
        
    forecast_df = pd.DataFrame(all_forecasts)
    return forecast_df, metrics

# Model C: Holt-Winters
def run_holt_winters(long_df):
    """Runs Holt-Winters, evaluates it, and generates a forecast."""
    print("\n--- Running Holt-Winters Model ---")
    
    # a. Evaluation
    predictions, actuals = [], []
    for part in long_df['Part No'].unique():
        part_df = long_df[long_df['Part No'] == part].set_index('Date')['Demand'].asfreq('MS')
        if len(part_df) < 25: continue
        train, test = part_df[:-3], part_df[-3:]
        try:
            model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add').fit()
            preds = model.forecast(len(test))
            predictions.extend(preds)
            actuals.extend(test)
        except Exception: continue

    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    metrics = {'RMSE': rmse, 'MAE': mae, 'R-squared': r2}
    
    # b. Forecasting
    all_forecasts = []
    for part in long_df['Part No'].unique():
        part_df = long_df[long_df['Part No'] == part].set_index('Date')['Demand'].asfreq('MS')
        forecast_values = np.array([0, 0, 0])
        try:
            if len(part_df) < 25: raise ValueError("Not suitable for Holt-Winters")
            model = ExponentialSmoothing(part_df, seasonal_periods=12, trend='add', seasonal='add').fit()
            forecast_values = model.forecast(3)
        except Exception:
            if not part_df.empty:
                moving_avg = part_df.tail(6).mean()
                forecast_values = np.array([moving_avg, moving_avg, moving_avg])
        
        total_forecast = sum(max(0, round(val)) for val in forecast_values)
        all_forecasts.append({'Part No': part, '3_Month_Forecast_HW': total_forecast})
        
    forecast_df = pd.DataFrame(all_forecasts)
    return forecast_df, metrics

# --- 3. Main Execution Block ---
if __name__ == "__main__":
    # Load and process data
    master_long_df = load_and_prepare_data()
    
    if master_long_df is not None:
        # Run all models
        xgb_forecast, xgb_metrics = run_xgboost(master_long_df.copy())
        sarima_forecast, sarima_metrics = run_sarima(master_long_df.copy())
        hw_forecast, hw_metrics = run_holt_winters(master_long_df.copy())
        
        # --- Generate Final Outputs ---
        
        # a. Performance Metrics DataFrame
        performance_df = pd.DataFrame({
            'XGBoost': xgb_metrics,
            'SARIMA': sarima_metrics,
            'Holt-Winters': hw_metrics
        }).T
        
        print("\n\n--- 🏆 Final Model Performance Comparison 🏆 ---")
        print(performance_df)
        
        # b. Save individual forecast files
        xgb_forecast.to_csv("forecast_xgboost.csv", index=False)
        sarima_forecast.to_csv("forecast_sarima.csv", index=False)
        hw_forecast.to_csv("forecast_holt_winters.csv", index=False)
        
        print("\n✅ Forecast files saved: 'forecast_xgboost.csv', 'forecast_sarima.csv', 'forecast_holt_winters.csv'")
        
        # c. Optional: Create a single combined forecast file
        combined_forecast = xgb_forecast.merge(sarima_forecast, on='Part No').merge(hw_forecast, on='Part No')
        combined_forecast.to_csv("forecast_combined.csv", index=False)
        print("✅ Combined forecast file saved: 'forecast_combined.csv'")



🔄 Step 1: Loading and preparing data...
✅ Data preparation complete.

--- Running XGBoost Model ---

--- Running SARIMA Model ---

--- Running Holt-Winters Model ---


--- 🏆 Final Model Performance Comparison 🏆 ---
                    RMSE        MAE  R-squared
XGBoost       156.493074  14.451050   0.836207
SARIMA         87.569655   8.316747   0.956565
Holt-Winters  129.819247  14.706610   0.887285

✅ Forecast files saved: 'forecast_xgboost.csv', 'forecast_sarima.csv', 'forecast_holt_winters.csv'
✅ Combined forecast file saved: 'forecast_combined.csv'


2025-07-24 18:09:09.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.433 No runtime found, using MemoryCacheStorageManager
2025-07-24 18:09:09.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 18:09:09.470 Thread 'MainThread':

In [3]:
pip install streamlit


  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/62/b1/44bd5f0eb1a6d9fa045db1e8bca77dc6751c12f7dacebf820ee708ea5acc/streamlit-1.47.0-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.5.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<7,>=4.0 from https://files.pythonhosted.org/packages/00/f0/2ef431fe4141f5e334759d73e81120492b23b2824336883a91ac04ba710b/cachetools-6.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<7,>=3.20 from https://files.pythonhosted.org/packages/44/3a/b15c4347dd4bf3a1b0ee882f384623e2063bb5cf9fa9d57990a4f7df2fb6